[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/ml-math-with-densworld/blob/main/modules/01-statistics-probability/notebooks/03-central-limit-theorem.ipynb)

# Lesson 3: The Central Limit Theorem

*"One mapmaker lies. Two mapmakers argue. Ten mapmakers approach the truth."*  
— Proverb of the Capital Survey Office

---

## The Core Problem

In the Dens, where the boundary between solid ground and densmuck shifts like a fever dream, mapmakers face an impossible task: survey terrain that refuses to stay still. Each individual survey contains error—from instruments, from weather, from fatigue, from the land itself.

Yet somehow, when the Capital Survey Office combines reports from fifteen different mapmakers at the same location, their averaged estimate is remarkably accurate. **How can averaging wrong answers give you the right one?**

The answer lies in the most important theorem in statistics: the **Central Limit Theorem** (CLT).

---

## Learning Objectives

By the end of this lesson, you will:
1. Understand why sample means follow a normal distribution—even when the data doesn't
2. Apply the CLT to quantify uncertainty in estimates
3. Know the √n rule: why 4× the data gives only 2× the precision
4. Recognize when CLT assumptions break down

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

# Nice plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Colab-ready data loading
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/ml-math-with-densworld/main/data/"

# Load the datasets
expeditions = pd.read_csv(BASE_URL + "expedition_outcomes.csv")
dens_boundary = pd.read_csv(BASE_URL + "dens_boundary_observations.csv")

print(f"Loaded {len(expeditions)} expedition records")
print(f"Loaded {len(dens_boundary)} boundary observations")

## Part 1: The Magic of Averaging

### One Mapmaker vs. Many

Imagine a single location on the Dens boundary—grid coordinate (50, 50). The true ground stability at this point is 0.65 (meaning 65% stable, 35% densmuck risk).

Mapmakers survey this point, but their measurements are noisy. Let's see what happens when we send one mapmaker vs. averaging ten:

In [ ]:
# Simulate mapmaker observations at a single point
TRUE_STABILITY = 0.65
MEASUREMENT_STD = 0.10  # Typical mapmaker precision

# Simulate 1000 scenarios where we send ONE mapmaker
n_simulations = 1000
single_observations = np.random.normal(TRUE_STABILITY, MEASUREMENT_STD, n_simulations)

# Simulate 1000 scenarios where we send TEN mapmakers and average
averaged_observations = []
for _ in range(n_simulations):
    ten_observations = np.random.normal(TRUE_STABILITY, MEASUREMENT_STD, 10)
    averaged_observations.append(ten_observations.mean())
averaged_observations = np.array(averaged_observations)

# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Single mapmaker
axes[0].hist(single_observations, bins=40, color='coral', edgecolor='black', alpha=0.7)
axes[0].axvline(TRUE_STABILITY, color='green', linewidth=3, linestyle='--', label=f'Truth = {TRUE_STABILITY}')
axes[0].axvline(single_observations.mean(), color='red', linewidth=2, label=f'Mean = {single_observations.mean():.3f}')
axes[0].set_xlabel('Observed Stability')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'One Mapmaker\nStd = {single_observations.std():.3f}')
axes[0].legend()
axes[0].set_xlim(0.2, 1.1)

# Ten mapmakers averaged
axes[1].hist(averaged_observations, bins=40, color='steelblue', edgecolor='black', alpha=0.7)
axes[1].axvline(TRUE_STABILITY, color='green', linewidth=3, linestyle='--', label=f'Truth = {TRUE_STABILITY}')
axes[1].axvline(averaged_observations.mean(), color='red', linewidth=2, label=f'Mean = {averaged_observations.mean():.3f}')
axes[1].set_xlabel('Averaged Stability Estimate')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Average of 10 Mapmakers\nStd = {averaged_observations.std():.3f}')
axes[1].legend()
axes[1].set_xlim(0.2, 1.1)

plt.tight_layout()
plt.show()

print(f"Precision improvement: {single_observations.std() / averaged_observations.std():.2f}x")
print(f"Theory predicts: √10 = {np.sqrt(10):.2f}x")

### The Key Insight

Notice two things:

1. **The average of 10 mapmakers is more concentrated around the truth** — the histogram is narrower
2. **The improvement is approximately √10 ≈ 3.16×** — not 10×!

This is the √n rule in action. More mapmakers help, but with diminishing returns.

## Part 2: The Central Limit Theorem Explained

The CLT states:

> If you take a sample of n observations from ANY distribution with mean μ and standard deviation σ, the distribution of the sample mean approaches a normal distribution with:
> - Mean = μ
> - Standard deviation = σ/√n

The remarkable part: **"any distribution."** It doesn't matter if the original data is skewed, bimodal, or weirdly shaped. The sample mean will still be approximately normal.

### Demonstration with Non-Normal Data

Let's prove this with creature market prices—which we know are heavily right-skewed:

In [ ]:
# Load creature market data (heavily skewed)
creature_market = pd.read_csv(BASE_URL + "creature_market.csv")
prices = creature_market['price_per_unit'].values

# Show the original skewed distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original distribution
axes[0, 0].hist(prices, bins=50, color='goldenrod', edgecolor='black', alpha=0.7)
axes[0, 0].set_title(f'Original Price Distribution\n(Skewness = {stats.skew(prices):.2f})')
axes[0, 0].set_xlabel('Price')

# Sample means for various n
sample_sizes = [2, 5, 10, 30, 100]
n_samples = 2000

for idx, n in enumerate(sample_sizes):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    
    sample_means = [np.random.choice(prices, size=n, replace=True).mean() for _ in range(n_samples)]
    sample_means = np.array(sample_means)
    
    axes[row, col].hist(sample_means, bins=40, color='steelblue', edgecolor='black', alpha=0.7, density=True)
    
    # Overlay theoretical normal
    theoretical_mean = prices.mean()
    theoretical_std = prices.std() / np.sqrt(n)
    x = np.linspace(sample_means.min(), sample_means.max(), 100)
    axes[row, col].plot(x, stats.norm.pdf(x, theoretical_mean, theoretical_std), 'r-', linewidth=2)
    
    axes[row, col].set_title(f'Sample Mean (n={n})\nSkewness = {stats.skew(sample_means):.2f}')
    axes[row, col].set_xlabel('Mean Price')

plt.suptitle('Central Limit Theorem: Skewed Data → Normal Sample Means', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("As sample size increases:")
print("  - The distribution of sample means becomes more symmetric")
print("  - The spread (standard deviation) decreases")
print("  - The shape approaches normal (red curve)")

### Mathematical Formulation

For a sample of n observations with sample mean $\bar{X}$:

$$\bar{X} \sim N\left(\mu, \frac{\sigma}{\sqrt{n}}\right)$$

Or equivalently, the standardized version:

$$Z = \frac{\bar{X} - \mu}{\sigma/\sqrt{n}} \sim N(0, 1)$$

The quantity $\sigma/\sqrt{n}$ is called the **Standard Error** of the mean.

## Part 3: The √n Rule in Practice

### How Much Data Do You Need?

The √n relationship has profound practical implications:

| To improve precision by... | You need... |
|---------------------------|-------------|
| 2× | 4× the data |
| 3× | 9× the data |
| 10× | 100× the data |

This is why the Capital Survey Office sends teams of mapmakers rather than waiting for one perfect measurement. Perfection is impossible; averaging is practical.

In [ ]:
# Demonstrate the √n relationship with real boundary data
true_mean = dens_boundary['observed_stability'].mean()  # Treat full dataset as "population"
true_std = dens_boundary['observed_stability'].std()

sample_sizes = [5, 10, 25, 50, 100, 200, 400]
n_simulations = 1000

observed_stds = []
theoretical_stds = []

for n in sample_sizes:
    sample_means = []
    for _ in range(n_simulations):
        sample = dens_boundary['observed_stability'].sample(n=n, replace=True)
        sample_means.append(sample.mean())
    observed_stds.append(np.std(sample_means))
    theoretical_stds.append(true_std / np.sqrt(n))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(sample_sizes, observed_stds, 'bo-', markersize=10, linewidth=2, label='Observed (simulation)')
ax.plot(sample_sizes, theoretical_stds, 'r--', linewidth=2, label='Theoretical (σ/√n)')

ax.set_xlabel('Sample Size (n)', fontsize=12)
ax.set_ylabel('Standard Error of Mean', fontsize=12)
ax.set_title('The √n Rule: More Data = Less Uncertainty\n(but with diminishing returns)', fontsize=14)
ax.legend(fontsize=11)
ax.set_xscale('log')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPrecision at different sample sizes:")
print(f"{'n':<10} {'Std Error':<15} {'Relative to n=5':<20}")
print("-" * 45)
base_se = theoretical_stds[0]
for n, se in zip(sample_sizes, theoretical_stds):
    print(f"{n:<10} {se:<15.4f} {base_se/se:.2f}x better")

### Practical Application: Survey Planning

The Capital Survey Office needs to estimate the average stability of a region to within ±0.02 (with 95% confidence). How many observations do they need?

In [ ]:
# Required sample size calculation
# 95% CI half-width = 1.96 * (σ/√n)
# We want: 1.96 * (σ/√n) ≤ 0.02
# So: n ≥ (1.96 * σ / 0.02)²

sigma = dens_boundary['measurement_error'].std()  # Use error std as proxy
desired_precision = 0.02
confidence_level = 0.95
z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)  # 1.96

required_n = (z_score * sigma / desired_precision) ** 2

print("Sample Size Planning for Boundary Survey")
print("=" * 50)
print(f"Measurement std (σ): {sigma:.4f}")
print(f"Desired precision: ±{desired_precision}")
print(f"Confidence level: {confidence_level:.0%}")
print(f"\nRequired sample size: n ≥ {required_n:.0f}")

# Show precision at various sample sizes
print(f"\n{'n':<10} {'95% CI half-width':<20} {'Meets target?':<15}")
print("-" * 45)
for n in [10, 25, 50, 100, 200, 500]:
    half_width = z_score * sigma / np.sqrt(n)
    meets = "✓" if half_width <= desired_precision else "✗"
    print(f"{n:<10} ±{half_width:<18.4f} {meets}")

## Part 4: Why the CLT Matters for Machine Learning

The Central Limit Theorem is the foundation for many statistical techniques:

### 1. Confidence Intervals
Because sample means are approximately normal, we can construct confidence intervals using the normal distribution.

In [ ]:
# Confidence interval for expedition success rate
success_rate = expeditions['success'].mean()
n = len(expeditions)

# For proportions, σ = √(p(1-p))
se = np.sqrt(success_rate * (1 - success_rate) / n)

ci_95 = (success_rate - 1.96 * se, success_rate + 1.96 * se)
ci_99 = (success_rate - 2.576 * se, success_rate + 2.576 * se)

print("Expedition Success Rate Estimation")
print("=" * 50)
print(f"Sample proportion: {success_rate:.1%}")
print(f"Sample size: {n}")
print(f"Standard error: {se:.4f}")
print(f"\n95% CI: ({ci_95[0]:.1%}, {ci_95[1]:.1%})")
print(f"99% CI: ({ci_99[0]:.1%}, {ci_99[1]:.1%})")

### 2. Hypothesis Testing
Because we know the distribution of sample means, we can test whether observed differences are statistically significant.

### 3. Regression Coefficients
In linear regression, the estimated coefficients are averages of sorts—and thus approximately normally distributed.

## Part 5: When the CLT Fails

The CLT requires certain conditions. It may fail when:

### 1. Sample Size Is Too Small
For highly skewed distributions, you may need n > 30 (or even n > 100) for the CLT to apply.

In [ ]:
# Demonstrate CLT failure with small samples from skewed data
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, n in enumerate([3, 10, 50]):
    sample_means = [np.random.choice(prices, size=n, replace=True).mean() for _ in range(2000)]
    
    axes[idx].hist(sample_means, bins=40, color='steelblue', edgecolor='black', alpha=0.7, density=True)
    
    # Try to fit normal
    theoretical_mean = np.mean(sample_means)
    theoretical_std = np.std(sample_means)
    x = np.linspace(min(sample_means), max(sample_means), 100)
    axes[idx].plot(x, stats.norm.pdf(x, theoretical_mean, theoretical_std), 'r-', linewidth=2)
    
    skewness = stats.skew(sample_means)
    axes[idx].set_title(f'n = {n}\nSkewness = {skewness:.2f}\n{"✓ CLT applies" if abs(skewness) < 0.5 else "⚠ Still skewed"}')
    axes[idx].set_xlabel('Sample Mean Price')

plt.suptitle('CLT Convergence Depends on Sample Size (and Original Distribution)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("For highly skewed data like prices:")
print("  - n=3: CLT definitely doesn't apply")
print("  - n=10: Marginal, still shows skewness")
print("  - n=50: CLT is reasonably applicable")

### 2. Infinite Variance (Fat Tails)

Some distributions have such heavy tails that the variance is technically infinite. The CLT doesn't apply to these. Fortunately, our Densworld data doesn't exhibit this extreme behavior.

### 3. Dependent Observations

The CLT assumes observations are independent. If mapmaker observations are correlated (e.g., they all survey on the same day with the same weather), the effective sample size is smaller than it appears.

In [ ]:
# Check for potential dependence: do observations cluster by weather?
weather_stability = dens_boundary.groupby('weather_conditions')['observed_stability'].agg(['mean', 'std', 'count'])

print("Stability Observations by Weather Condition")
print("=" * 50)
print(weather_stability.round(3))
print(f"\nIf observations are clustered by weather, they may not be independent.")
print(f"This could make our CLT-based confidence intervals too narrow.")

## Part 6: Practical Exercise — Combining Mapmaker Surveys

Let's simulate what the Capital Survey Office does: combine multiple mapmaker observations to estimate true stability at a location.

In [ ]:
# Filter to a single approximate location (grid cell)
location_data = dens_boundary[
    (dens_boundary['location_x'].between(45, 55)) & 
    (dens_boundary['location_y'].between(45, 55))
]

print(f"Found {len(location_data)} observations in grid cell (45-55, 45-55)")
print(f"\nObservers who surveyed this area:")
print(location_data['observer_name'].value_counts())

# Calculate group estimate and confidence interval
stability_values = location_data['observed_stability']
group_mean = stability_values.mean()
group_se = stability_values.std() / np.sqrt(len(stability_values))

print(f"\nCombined estimate (CLT-based):")
print(f"  Mean stability: {group_mean:.3f}")
print(f"  Standard error: {group_se:.4f}")
print(f"  95% CI: ({group_mean - 1.96*group_se:.3f}, {group_mean + 1.96*group_se:.3f})")

# Compare individual mapmaker estimates
print(f"\nIndividual mapmaker estimates:")
individual = location_data.groupby('observer_name')['observed_stability'].mean()
for name, value in individual.items():
    print(f"  {name}: {value:.3f}")

## Summary

| Concept | Key Insight | Formula |
|---------|-------------|--------|
| Central Limit Theorem | Sample means are normal (regardless of original distribution) | $\bar{X} \sim N(\mu, \sigma/\sqrt{n})$ |
| Standard Error | Uncertainty of sample mean decreases with √n | $SE = \sigma / \sqrt{n}$ |
| √n Rule | 4× data → 2× precision | Precision ∝ √n |
| Sample Size Planning | Required n = (z × σ / precision)² | Solve for n |
| CLT Requirements | Large n, finite variance, independence | Rules of thumb vary |

---

## Exercises

### Exercise 1: Bootstrap vs. CLT

For the expedition `catch_value` variable:
1. Calculate a 95% confidence interval using the CLT (normal approximation)
2. Calculate a 95% confidence interval using bootstrap (10,000 resamples)
3. How different are they? Why?

In [ ]:
# Exercise 1: Your code here
# CLT-based CI: mean ± 1.96 * (std / sqrt(n))
# Bootstrap CI: np.percentile(bootstrap_means, [2.5, 97.5])


### Exercise 2: Sample Size for Creature Market Analysis

A merchant wants to estimate the average price of `mammal` category creatures to within ±5 gold pieces with 95% confidence. Based on the current data, how many sales records would they need?

In [ ]:
# Exercise 2: Your code here
# First, calculate the standard deviation of mammal prices
# Then use: n = (1.96 * sigma / precision)^2


### Exercise 3: Testing CLT Assumptions

Check whether the `casualties` variable from expeditions data satisfies CLT requirements:
1. Calculate skewness and kurtosis
2. Simulate the distribution of sample means for n=5, n=20, n=50
3. At what sample size does the distribution become approximately normal?

In [ ]:
# Exercise 3: Your code here


### Exercise 4: Comparing Mapmaker Precision

Using the `dens_boundary` data:
1. Group observations by `instrument_type`
2. Calculate the standard error of `observed_stability` for each instrument type
3. How many observations with `pacing` would you need to match the precision of 10 observations with `theodolite`?

In [ ]:
# Exercise 4: Your code here


---

## Next Lesson

In **Lesson 4: Hypothesis Testing**, we'll learn how to determine whether observed differences are real or just random noise. We'll analyze the Scholar Debates to ask: Do Stone School philosophers really win more debates, or is it just luck?

*"Correlation is not causation. The Stone School's victories may say more about the judges than the philosophy."*  
— Yasho Krent, in defense at the Senate Inquiry